SwatGrains 2020 Data Analysis: BondInfo.txt file - Contact Networks and Force Distribution
==============

## 0.1. Load Data

Loads a set of bondInfo.txt files as hierarchically indexed DataFrames, which contain each jammed trial indexed by trial name and original row number. DataFrames are stored in the dictionary `dataDict`, and are indexed by abbreviated file names for easier iteration.
In order to identify which trials are jammed, a Pressures.txt file has to be generated for each data file via Info_Analysis.ipynb.

This cell also defines the `sort_particles()` function, which takes bond data and translates it into a DataFrame of particles identified by location and particle type.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp
import matplotlib
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook

def get_pressures(file):
    # the Pressures.txt file (created in Info_Analysis.ipynb) contains the seeds of all jammed trials, which we use to filter our data.
    # for some of our analysis, we'll also need pressures corresponding to each trial, which this file gives us.
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('BondInfo','Pressures')
    trialPressures=pd.read_csv('/'.join(filePath), sep=' ', index_col=0)['pressure']
    if file[-8]=='6':
        trialPressures=trialPressures/4
    return trialPressures

def read_data(filename):
    dataset=pd.read_csv(filename, usecols=range(7), sep=' ', header=0, names=['x1','y1','x2','y2','energy','force','bondType']) # read bondInfo.txt
    dataset['trialName']=dataset.groupby((dataset.x1=='>>>>>').cumsum())['y1'].transform(lambda x: x.max()) # group by trial dividers, recognized by >>>>>, and label each row with group name
    dataset = dataset[dataset.x1 != '>>>>>'].reset_index(drop=True) # remove trial dividers

    # Generate higher level index
    dataset.index.rename('rowNumber', inplace=True)
    dataset.set_index('trialName',append=True,inplace=True)
    dataset.index=dataset.index.swaplevel()
    dataset=dataset.astype('float32') # convert all columns to float32 - 7 decimal places of precision is plenty
    dataset['bondType']=dataset['bondType'].astype('int16') # except bond type, which is a column of integers
    dataset['normalizedForce']=dataset.groupby('trialName')['force'].transform(lambda x: x/x.mean())
    dataset[['x1','y1','x2','y2','normalizedForce']]=(1e6*dataset[['x1','y1','x2','y2','normalizedForce']]).astype('int')
    return dataset.loc[get_pressures(filename).index]


def sort_particles(dataset, types=[0,1,2]):
    # same function as is used in cell 2.1 - sort particles into types via bondType column.
    # returns a dataframe of every coordinate in bondInfo - duplicates included.
    small_mobile=pd.DataFrame()
    large_mobile=pd.DataFrame()
    pins=pd.DataFrame()

    for bondGroup in dataset.groupby('bondType'):
        coords1=bondGroup[1][['x1','y1']].rename(columns={'x1':'x','y1':'y'})
        coords2=bondGroup[1][['x2','y2']].rename(columns={'x2':'x','y2':'y'})
        # case-by-case for each bond type
        if bondGroup[0]==0:
            # small-small
            small_mobile=pd.concat([small_mobile,coords1,coords2])

        if bondGroup[0]==1:
            # small-large
            small_mobile=pd.concat([small_mobile,coords1])
            large_mobile=pd.concat([large_mobile,coords2])

        if bondGroup[0]==2:
            # large-large
            large_mobile=pd.concat([large_mobile,coords1,coords2])

        if bondGroup[0]==3:
            # pin-small
            small_mobile=pd.concat([small_mobile,coords2])
            pins=pd.concat([pins,coords1])

        if bondGroup[0]==4:
            # pin-large
            large_mobile=pd.concat([large_mobile,coords2])
            pins=pd.concat([pins,coords1])
    # Cut off coordinate values at six decimals - this accounts for issues with floating point arithmetic
    large_mobile=large_mobile.round(6)
    small_mobile=small_mobile.round(6)
    pins=pins.round(6)

    sortedFrame=pd.concat([large_mobile.assign(particleType=2),small_mobile.assign(particleType=1), pins.assign(particleType=0)])
    return  sortedFrame[sortedFrame.particleType.isin(types)] # return combined dataframes, with additional column for type

filenames={'non0_07-11':'simulation_data/non_BondInfo07-11.txt', 'non0_07-13':'simulation_data/non_BondInfo07-13.txt',
           'non0_08-02':'simulation_data/ZeroPinData/non_BondInfo08-02.txt', 'non0_06-20':'simulation_data/ZeroPinData/non_BondInfo06-20.txt',
           'squ16_08-05':'simulation_data/squ_BondInfoNp16_08-05.txt', 'squ64_08-05':'simulation_data/squ_BondInfoNp64_08-05.txt','squ36_07-11':'simulation_data/squ_BondInfo07-11.txt',
           'squ64_07-12':'simulation_data/squ_BondInfo07-12.txt', 'squ64_07-13':'simulation_data/squ_BondInfo07-13.txt',
           'tri36_07-11':'simulation_data/tri_BondInfo07-11.txt', 'tri64_08-08':'simulation_data/tri_BondInfoNp64_08-08.txt', 'tri64_07-12':'simulation_data/tri_BondInfo07-12.txt',
           'ran36_07-11':'simulation_data/ran_BondInfo07-11.txt', 'squ100_07-19':'simulation_data/squ_BondInfo07-19.txt', 'tri100_07-19':'simulation_data/tri_BondInfo07-19.txt', 'ran100_07-19':'simulation_data/ran_BondInfo07-19.txt'}

files = ['non0_07-11','squ36_07-11', 'squ64_07-12', 'squ100_07-19','ran36_07-11','ran100_07-19', 'tri100_07-19']
dataDict={file:read_data(filenames[file]) for file in tqdm_notebook(files)}

In [ ]:
dataDict['non0_07-11'].index.get_level_values(0).unique()

# 1. Analysis on Entire File

## 1.1. Contact numbers
Creates the DataFrame `dfContacts`, which identifies each particle with its corresponding number of contacts, indexed by particle type (pin, small, large).

In [ ]:
contactsDict={file:sort_particles(dataDict[file]).groupby(['x','y','particleType','trialName']).size().reset_index(name='contacts').set_index('particleType') for file in tqdm_notebook(files)} # dataframe of particle coordinates, with contact number and trial labeled

## 1.2. Extrapolate $z_\text{iso}$ as pressure goes to 0
This cell fits $z(P)$ to a square root function. We expect $\Delta z\propto\sqrt{\Delta\phi},$ and by extension
$$z(P)=c\sqrt{P}+z_\text{iso},$$
where $c$ is (we hope) some unit-dependent constant.

In [ ]:
import re
def power_fit(xdata, coeff, intercept):
    return coeff*np.sqrt(xdata)+intercept

colormap = matplotlib.cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 'r', '#FFFFFF', '#FFFFFF']) # classic
#colormap = matplotlib.cycler('color',['#FAD55C','#68C182','#267DB3','#ED6647','#8561C8','#000000', '#fef9e6', '#c0dff2']) # blue-yellow
plt.rc('axes',prop_cycle=colormap)


def get_pressures(file):
    # for some of our data, we'll need pressures corresponding to each trial, which is found in Pressures files generated by jamming_plots.ipynb.
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('BondInfo','Pressures')
    trialPressures=pd.read_csv('/'.join(filePath), sep=' ', index_col=0)['pressure']
    if file[-8]=='6':
        trialPressures=trialPressures/4
    return trialPressures
intercepts=[]
plt.figure(figsize=(8,8))
for colorCounter, file in tqdm_notebook(enumerate(files)):
    meanContacts=contactsDict[file].loc[[1,2]].groupby('trialName')['contacts'].mean()
    pressureData = pd.concat([get_pressures(filenames[file]),meanContacts], axis=1)
    pressureData = pressureData[pressureData.pressure>1e-8]
    pressureData.index.name='trialName'
    [coeff,intercept], covariance = sp.curve_fit(power_fit, pressureData['pressure'], pressureData['contacts'])
    perror=np.sqrt(np.diag(covariance))
    plt.plot(pressureData['pressure'], pressureData['contacts'],'C{}.'.format(colorCounter), alpha=0.2)
    xValues=np.linspace(5e-4,1.35,10000)
    yValues=power_fit(xValues, coeff, intercept)
    plt.plot(xValues,yValues, 'C{}'.format(colorCounter), alpha=0.7, label='{name}, $z_c={z:.3f}+/-{err:.3f}$'.format(name=file, z=intercept,err=perror[1]))
    intercepts.append(intercept)

    # Export delta z values to Pressures.txt file for plotting in jamming_plots.ipynb
    #pressurePath=filenames[file].split('/')
    #pressurePath[-1]=pressurePath[-1].replace('BondInfo','Pressures')
    #pressureData['delta_z']=pressureData['contacts']-intercept
    #pressureData.drop('contacts', axis=1, inplace=True)
    #pressureData.to_csv('/'.join(pressurePath), sep=' ')

    # Save fit function values for aggregate analysis in Cell 1.3
    #parameters = pd.DataFrame({'coeff': coeff, 'intercept': intercept, 'err1': perror[0], 'err2': perror[1]}, index=[file])
    #print(parameters)
    #parameters.to_csv('simulation_data/contact_number_parameters.txt', mode = 'a', sep = ' ', header = False)

plt.ylabel('mean contact number', fontsize=14)
plt.xlabel('pressure', fontsize=14)
#plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.title('z vs. pressure for 230 particle square lattices', fontsize=16)#.format(num=int(dfContacts.groupby('trialName').size().mean()),z=intercept, err=perror[1]))
plt.savefig('plots/presentation/z_vs_p-squares.png', dpi=300)
plt.show()

#n_f=[0,0.0696,0.2783]
#plt.plot(n_f,intercepts,'C0.')
#plt.ylabel('critical contact number')
#plt.xlabel('pin density')
#plt.title('$z_c$ vs $n_f$')
#plt.savefig('plots/zc_vs_nf-non_corrected-comparison.png', dpi=300)
#plt.show()

## 1.3. Standalone: Contact Number Summary
Since we're pulling data from the contact_number_parameters.txt file exclusively, this cell contains the same imports as the first cell and the power fit function from the last cell and runs without any prior initialization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp
import matplotlib
import pandas as pd
import os
import seaborn as sns

def power_fit(xdata, coeff, intercept):
    return np.sqrt(coeff*xdata)+intercept


contactParameters = pd.read_csv('simulation_data/contact_number_parameters.txt', sep=' ', header=None, names=['trial','coefficient','z_iso','err1','err2'])
#xValues=np.linspace(0,1.5,100)
#yList=[0,0,0,0,36,64,36,64]
#for row in contactParameters.iterrows():
#    yList.append((row[1]['num_pins'],power_fit(xValues, row[1]['coefficient'],row[1]['z_iso'])))

fig = plt.figure(figsize=(18,6))

num_pins_array=np.array([0,0,0,0,36,64,36,64])
ax2 = fig.add_subplot(1,3,1)
ax2.margins(0.15)
plt.xlabel('number of pins')
plt.ylabel('$z_{iso}$')
plt.title('isostatic number $z_{iso}$ vs. number of pins')

ax3 = fig.add_subplot(1,3,2)
ax3.margins(0.15)
plt.xlabel('number of pins')
plt.ylabel('$c$')
plt.title('coefficient $c$ vs. number of pins')


ax1 = fig.add_subplot(1,3,3)
ax1.margins(0.15)
plt.xlabel('number of particles')
plt.ylabel('$c$')
plt.title('coefficient $c$ vs. number of particles')

for row, num_pins, num_particles in zip(contactParameters.iterrows(), num_pins_array, [230,230,230,920,230,230,230,230]):
    if row[1]['trial'][:3]=='non':
        markerstyle='X'
    elif row[1]['trial'][:3]=='squ':
        markerstyle='s'
    elif row[1]['trial'][:3]=='tri':
        markerstyle='^'
    ax2.errorbar(num_pins, row[1]['z_iso'], yerr=row[1]['err2'], fmt='.', ms=8, alpha=0.6, marker=markerstyle, label=row[1]['trial'])
    ax3.errorbar(num_pins, row[1]['coefficient'], yerr=row[1]['err1'], fmt='.',  ms=8, alpha=0.6, marker=markerstyle, label=row[1]['trial'])
    ax1.errorbar(num_particles, row[1]['coefficient'], yerr=row[1]['err1'], fmt='.',  ms=8, alpha=0.6, marker=markerstyle, label=row[1]['trial'])

plt.legend()
plt.tight_layout()
#plt.savefig('plots/contact_number_trends_new.png', dpi=300)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp
import matplotlib
import pandas as pd
import os
import seaborn as sns
matplotlib.rcParams['text.usetex'] = True

contactParameters = pd.read_csv('simulation_data/contact_number_parameters.txt', sep=' ', header=None, names=['trial','coefficient','z_iso','err1','err2', 'lambda'])
contactParameters['w_pins']=[0,0,0,0,36/266,64/294,36/266,64/294, 16/246, 64/294,64/294, 36/266,100/330,100/330,100/330]
plt.figure(figsize=(8,8))
contactParameters['geo']=contactParameters['trial'].transform(lambda x: x[:3])
#contactParameters.drop(contactParameters[contactParameters.geo=='non'].index, inplace=True)
for colorCounter, geometry in enumerate(contactParameters.groupby('geo'),0):
    if geometry[0]=='non':
        marker='X'
    elif geometry[0]=='squ':
        marker='s'
    elif geometry[0]=='tri':
        marker='^'
    elif geometry[0]=='ran':
        marker='d'
    plt.errorbar(geometry[1]['w_pins'], geometry[1]['z_iso'], yerr=geometry[1]['err2'], fmt=marker, alpha=0.6, label=geometry[0], color='C{}'.format(colorCounter))

xValues=np.linspace(*plt.xlim(), 1000)
yValues=4*(1-xValues)/(1-xValues/2)
plt.plot(xValues,yValues, label='theoretical expectation', color='C4')
plt.figtext(0.25,0.4,r'$z_c(w)=2d\displaystyle\frac{1-w}{1-w/2}$', fontsize=17)
plt.legend(fontsize = 14)
plt.ylabel('$z_c$', fontsize = 20)
plt.xlabel('$w_{pins}$', fontsize = 20)
plt.title('critical $z$ vs $w_{pins}$', fontsize = 23)
#plt.savefig('plots/presentation/z_c_vs_wpins-comparison.png', dpi=300)

matplotlib.rcParams['text.usetex'] = False

## 1.4. (Deprecated). Finding contact number with NetworkX
After failing to effectively plot topology with NetworkX, we revisited the module, attempting to use its graph.neighbors() function to find contact numbers without too much fuss. Unfortunately, NetworkX is not well-suited to such large data sets, and this script, while theoretically functional, ends up being incredibly inefficient and has never finished running.

## Examining $N_{excess}$

In [ ]:
#import matplotlib
#matplotlib.rcParams.update(matplotlib.rcParamsDefault)
from scipy import stats
# What's really important: picking a fun color scheme
#colormap = matplotlib.cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 'r', '#FFFFFF', '#FFFFFF']) # classic
#colormap = matplotlib.cycler('color',['#FAD55C','#68C182','#267DB3','#ED6647','#8561C8','#000000', '#fef9e6', '#c0dff2']) # blue-yellow
#plt.rc('axes',prop_cycle=colormap)

fig, (axRan, axSqu, axTri) = plt.subplots(3,1,figsize=(8,10))
for colorCounter, file in enumerate(tqdm_notebook(files)):
    N_pp=dataDict[file][dataDict[file].bondType<=2].groupby(level=0).size()
    N_pf=dataDict[file][dataDict[file].bondType>=3].groupby(level=0).size()
    num_particles=sort_particles(dataDict[file], types=[1,2]).drop_duplicates(keep='first').groupby('trialName').size()
    N_excess=N_pp+N_pf-2*num_particles-1
    N_pf=N_pf[N_excess>0]
    N_excess=N_excess[N_excess>0]
    regression=stats.linregress(N_pf,N_excess)
    yValues=np.array([0,50])
    xValues=(yValues-regression[1])/regression[0]
    if file[:3]=='ran':
        axRan.plot(N_pf, N_excess, 's', ms=2, label='{}, slope={slope:.2f}, r={r:.2f}'.format(file, slope=regression[0], r=regression[2]), alpha=0.6, color='C{}'.format(colorCounter))
        axRan.plot(xValues,yValues, color='C{}'.format(colorCounter))
    elif file[:3]=='squ':
        axSqu.plot(N_pf, N_excess, 's', ms=2, label='{}, slope={slope:.2f}, r={r:.2f}'.format(file, slope=regression[0], r=regression[2]), alpha=0.6, color='C{}'.format(colorCounter))
        axSqu.plot(xValues,yValues, color='C{}'.format(colorCounter))
    elif file[:3]=='tri':
        axTri.plot(N_pf, N_excess, 's', ms=2, label='{}, slope={slope:.2f}, r={r:.2f}'.format(file, slope=regression[0], r=regression[2]), alpha=0.6, color='C{}'.format(colorCounter))
        axTri.plot(xValues,yValues, color='C{}'.format(colorCounter))

for axis in [axRan,axSqu,axTri]:
    axis.set(aspect='equal',adjustable='datalim')
    axis.set_xlim(0,95)
    axis.set_ylim(0,50)
    axis.legend(loc='upper left')

plt.xlabel('$N_{pf}$')
axSqu.set_ylabel('$N_{excess}$')
fig.suptitle('$N_{excess}$ vs particle pin contacts', y=0.93)
#plt.savefig('plots/Nexcess_vs_Npf-July20-regressed.png', dpi=300)
plt.show()

# 2. Finding Neighbors

## 2.1. Find $n_6$ of Large Neighbors
Iterates through every trial identifying the number of large neighbors in n_6 for each large particle
Returns a series

In [ ]:
from scipy import spatial

def get_large_neighbors(particles):
    # Uses scipy.spatial.KDTree to identify nearest six neighbors, then returns array of number of large neighbors for each large particle

    # Initialize data
    particles=particles.drop_duplicates(keep='first') # just want one instance of each particle, rather than once per contact
    particles=particles[particles.particleType>0] # remove pins
    tree=spatial.KDTree(list(zip(particles['x'],particles['y']))) # load coordinates as KDTree
    large_particles=particles[particles.particleType==2] # series of large particles, which we iterate over
    large_neighbors=np.array([],dtype='int16') # array to add number of neighbors for each particle to (within the trial)

    # Find neighbors of each particle
    try:
        for neighbor_index in tree.query(list(zip(large_particles['x'],large_particles['y'])),k=7)[1]: # Get indices of six closest particle neighbors (seven including self)
            try:
                neighborTypes=particles.iloc[neighbor_index].groupby('particleType').size() # count how many neighbors are large
                large_neighbors=np.append(large_neighbors,neighborTypes[2]) # add large neighbor count to array

            except IndexError: # Error case where a data set has fewer than six particles
                raise IndexError

        return large_neighbors

    except ValueError: # Error case where a data set has no large particles
        raise ValueError

def write_neighbors(neighbor_column):
    # Writes series of large neighbors as a new column in large_neighbors.txt for future reference

    # Load text file
    try:
        data = pd.read_csv('simulation_data/new_large_neighbors.txt', sep=' ', dtype='Int16')
    except:
        print('Note: simulation_data/new_large_neighbors.txt was empty or missing.')
        data=pd.DataFrame(dtype='Int16')

    data=pd.concat([data,neighbor_column], axis=1) # add new column to existing data
    data=data.loc[:,~data.columns.duplicated(keep='last')] # in case of duplicates, overwrite
    data.to_csv('simulation_data/new_large_neighbors.txt', sep = ' ', index=False) # write new data back to text file
    return data

def write_n_6(n6_column, file):
    # Writes series of n_6 values as a new column in corresponding Pressures file

    # Load text file
    filePath=file.split('/')
    filePath[-1]=filePath[-1].replace('BondInfo','Pressures')
    try:
        data = pd.read_csv('/'.join(filePath), sep=' ', index_col=0)
    except:
        print('Note: {} was empty or missing.'.format('/'.join(filePath)))
        data=pd.DataFrame()

    data['n_6']=n6_column # add new column to dataframe - this method of creating a column skips any values in n6_column whose trial names aren't in Pressures.txt (not actually jamming)
    data.index.name='trialName'
    data.to_csv('/'.join(filePath), sep = ' ') # write new data back to text file
    return data

for file in files:
    print('Processing:', file)
    # Find number of large particle neighbors for all trials in BondInfo (will take one or two minutes)
    large_neighbors=np.array([], dtype=np.int16) # initialize array for number of neighbors for entire BondInfo file
    n_6=[]
    for sorted_trial in tqdm_notebook(sort_particles(dataDict[file]).groupby('trialName')):
        try:
            trial_neighbors=get_large_neighbors(sorted_trial[1])
            large_neighbors=np.append(large_neighbors, trial_neighbors-1) # add array of large neighbor values for this trial to the overall array
            n_6.append([sorted_trial[0],trial_neighbors[trial_neighbors==7].size/trial_neighbors.size]) # get n_6, percentage of large particles with six large neighbors
        except:
            print('EXCEPTION ON TRIAL', sorted_trial[0]) # this warns us if we can't get large neighbors for a trial, and skips over it

    # Export neighbor data
    n_6=np.asarray(n_6) # list to array
    n_6=pd.Series(n_6[:,1],index=n_6[:,0]) # array to series
    write_neighbors(pd.Series(large_neighbors, name=file, dtype='Int16'))
    write_n_6(n_6, filenames[file])

## 2.2. Creates a neighbors 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sp
import matplotlib
import pandas as pd
import os
import seaborn as sns

def read_neighbors(filename):
    data = pd.read_csv(filename, sep=' ').astype('Int16')
    return data

neighbors = read_neighbors('simulation_data/new_large_neighbors.txt')
neighbors = neighbors.reindex(sorted(neighbors.columns), axis=1)
plt.figure(figsize=(10,6))
for label, content in neighbors.items():
    sns.distplot(content.dropna(), hist=True, kde=True, bins=np.linspace(-0.5,6.5,8), hist_kws={'histtype':'step','linewidth':1,'alpha':0.8}, kde_kws={'linewidth': 0},
                 label='{name}, $n_6={n:.4f}$'.format(name=label, n=content[content==6].size/content.dropna().size))

plt.ylim(0,0.35)
plt.ylabel('Probability Density')
plt.xlabel('Number of Large Neighbors')
plt.title('Normalized Distribution of Closest Six Neighbors of Large Particles')
plt.legend()
#plt.savefig('plots/n_6_histogram-comparison.png', dpi=300)
plt.show()

In [ ]:
neighborDensities=pd.DataFrame(dtype='float32')
for trial in neighbors:
    neighborDensities[trial]=neighbors.groupby(trial).size()/neighbors[trial].dropna().size

neighborDensities['mean']=neighborDensities.mean(axis=1)
neighborDensities=neighborDensities.transform(lambda x: x-x['non0'], axis=1)
neighborDensities['left']=neighborDensities.index-0.5
neighborDensities['right']=neighborDensities.index+0.5
xValues=pd.concat([neighborDensities['left'],neighborDensities['right']]).sort_index()

plt.figure(figsize=(8,6))
for trial in neighbors:
    plt.plot(xValues,neighborDensities[trial].repeat(2), label=trial, alpha=0.8)

plt.title('Closest Six Neighbors of Large Particles, normalized and zeroed to non0')
plt.xlabel('Number of Large Neighbors')
plt.legend()
#plt.savefig('plots/n_6_zeroed-comparison.png', dpi=300)
plt.show()

In [ ]:
plotGrid=[('squ_PressuresNp16_08-05.txt', 'squ16'), ('squ_PressuresNp64_08-05.txt', 'squ64'), ('tri_PressuresNp64_08-08.txt', 'tri64'), ('ZeroPinData/non_Pressures08-02.txt', 'non0')]
fig=plt.figure(figsize=(8,8))
for counter, (file, name) in enumerate(plotGrid, 1):
    fig.add_subplot(2,2,counter)
    n6_data=pd.read_csv('simulation_data/'+file, sep=' ', index_col=0)
    plt.plot(n6_data['pressure'], n6_data['n_6'], 'C{}.'.format(counter-1), alpha=0.3)
    plt.xlim(-0.1,2.2)
    plt.ylim(-0.01,0.15)
    plt.title(name)
#plt.legend()
fig.suptitle('n_6 vs. pressure', fontsize=14)
fig.text(0.5, 0.06, 'pressure', ha='center')
fig.text(0.04, 0.5, '$n_6$', va='center', rotation='vertical')
plt.subplots_adjust(top=0.9)
#plt.savefig('plots/n_6_vs_pressure-comparison.png', dpi=300)

## 1.5. Force distribution
Plots histograms of the normalized contact force, first for the entire data set, then broken up by bond type, then normalized as a kernel density estimate.

In [ ]:
colormap = matplotlib.cycler('color',['#FAD55C','#68C182','#267DB3','#ED6647','#8561C8','#000000', '#fef9e6', '#c0dff2']) # blue-yellow
plt.rc('axes',prop_cycle=colormap)

fig2 = plt.figure(figsize=(12,18))
hist1=fig2.add_subplot(3,1,1)
sns.distplot(entireFile['normalizedForce']/1e6, hist=True, kde=False,
             bins=1000, color = 'darkblue',
             hist_kws={'histtype':'step','linewidth':1,'alpha':0.95},
             )
hist1.set_xlim(-0.15,5.5)
hist1.set_title('force distribution for square lattice with $n_f={0.157}$, 07-11 2020')
hist1.set_xlabel('')
hist1.set_ylabel('counts')
hist1.set_yscale('log')

hist2=fig2.add_subplot(3,1,2)
entireFile.groupby('bondType')['normalizedForce'].apply(lambda x: sns.distplot(x/1e6, hist=True, kde=False, bins=1000, hist_kws={'histtype':'step','linewidth':1,'alpha':0.95}, label='bondType {}'.format(x.name)))
hist2.set_title('distribution by bond type')
hist2.set_xlabel('')
hist2.set_ylabel('counts')
hist2.set_yscale('log')
hist2.set_xlim(-0.15,5.5)
hist2.legend()

hist3=fig2.add_subplot(3,1,3)
entireFile.groupby('bondType')['normalizedForce'].apply(lambda x:sns.kdeplot(x/1e6,label='bondType {}'.format(x.name)))
hist3.set_title('probability density KDEs by bond type (linear axes)')
hist3.set_xlabel('normalized contact forces')
hist3.set_ylabel('probability density')
hist3.set_xlim(-0.15,5.5)
#hist3.set_yscale('log')
hist3.legend()
#plt.savefig('plots/07-11-squ36-force_distribution_histogram.png', dpi=300)
plt.show()

In [ ]:
from scipy import stats
plt.figure(figsize=(10,6))
for file, label in zip(['squ100_07-19','squ36_07-11','non0_07-11'],['100 Pins', '36 Pins', '0 Pins']):
    bw = 2*stats.iqr(dataDict[file][dataDict[file].bondType<=2]['normalizedForce']/1e6, rng=(25,75), scale=1.0, nan_policy='omit')/(len(dataDict[file])**(1/3))
    plt.hist(dataDict[file][dataDict[file].bondType<=2]['normalizedForce']/1e6, bins=int((dataDict[file][dataDict[file].bondType<=2]['normalizedForce'].max()/1e6-dataDict[file][dataDict[file].bondType>=3]['normalizedForce'].min()/1e6)/bw), density=True, alpha=0.4, label=label)
#plt.yscale('log')
plt.xlim(0,2)
plt.legend()
plt.xlabel('normalized force')
plt.ylabel('probability density')
plt.title('Contact Force Distributions, Peak Behavior')
#plt.savefig('plots/presentation/forceDistribution-peak.png', dpi=300)

In [ ]:
dataDict['non0_07-11'][dataDict['non0_07-11'].isna().any(axis=1)]

## Force Network Analysis

In [ ]:
from scipy import stats

def linear_fit(x, beta, intercept):
    return beta*x+intercept

bw=0.08
fig=plt.figure(figsize=(20,16))
locations=[1,2,4,5,7,3,6,9]
for location, file in zip(locations,files):
    fig.add_subplot(3,3,location)
    n, bins = plt.hist(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce']/1e6, bins=int(1e-6*dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce'].max()/bw), histtype='step',alpha=0.6, density=True, label='particle-particle')[:2]
    bins=bins[:-1]+np.diff(bins)/2
    particleparticle=pd.DataFrame({'normalizedForce':bins,'density':n})
    particleparticle=particleparticle[(particleparticle.normalizedForce>3)&(particleparticle.density>0)]#&(particleparticle.normalizedForce<5)]
    particleparticle['density']=np.log(particleparticle['density'])
    particleregress = stats.linregress(particleparticle['normalizedForce'], particleparticle['density'])
    RMS_Particle=np.sqrt(sum((particleparticle['density']-np.exp(linear_fit(particleparticle['normalizedForce'],particleregress[0],particleregress[1])))**2)/len(particleparticle))
    if file[:4] != 'non0':
        n, bins = plt.hist(dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce']/1e6, bins=int(1e-6*dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce'].max()/bw), histtype='step',alpha=0.6, density=True, label='particle-pin')[:2]
        bins=bins[:-1]+np.diff(bins)/2
        particlepin=pd.DataFrame({'normalizedForce':bins,'density':n})
        particlepin=particlepin[(particlepin.normalizedForce>3)&(particlepin.density>0)]#&(particlepin.normalizedForce<5)]
        particlepin['density']=np.log(particlepin['density'])
        pinregress = stats.linregress(particlepin['normalizedForce'], particlepin['density'])
        RMS_Pin=np.sqrt(sum((particlepin['density']-np.exp(linear_fit(particlepin['normalizedForce'],pinregress[0],pinregress[1])))**2)/len(particlepin))
    if file[:3]=='ran':
        plt.xlim(-20,210)
#    if file[:3]=='squ':
#        plt.xlim(-5,40)
#    if file[:3]=='tri':
#        plt.xlim(-5,49)
    #plt.xlim(-0.4,15)
    plt.ylim(3e-5,3)
    xValues=np.array(plt.xlim())
    plt.plot(xValues, np.exp(linear_fit(xValues, particleregress[0], particleregress[1])), 'C0', label='slope = {slope:.4f}, r={r:.4f}\n $\chi^2$={rms:.4f}, stderr={s:.3f}'.format(slope=particleregress[0],rms=RMS_Particle, s = particleregress[4], r=particleregress[2]))
    if file[:4]!='non0':
        plt.plot(xValues, np.exp(linear_fit(xValues, pinregress[0],pinregress[1])), 'C1', label='slope = {slope:.4f}, r={r:.4f}\n $\chi^2$={rms:.3f}, stderr={s:.3f}'.format(slope=pinregress[0],rms=RMS_Pin, s=pinregress[4], r=pinregress[2]))
    plt.title(file)
    plt.yscale('log')
    plt.xlabel('normalized force')
    plt.ylabel('probability density')
    plt.legend()
fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Force Distribution for Particles vs. Pins, 230 Particles, Regressed for 3<normalizedForce, July 2020 Data', x=0.5,y=0.95, fontsize=16) # title of entire figure
#plt.savefig('plots/force_distribution_particle_pin-comparison-regressed7_23.png', dpi=300)

In [ ]:
bw=0.08
fig, ((axParticle36,axPins36),(axParticle64,axPins64)) = plt.subplots(2,2, figsize=(15,12))
for file in ['squ36_07-11','tri36_07-11','ran36_07-11','non07-11']:
    sns.distplot(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce']/1e6, bins=int(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce'].max()/bw), hist=True, kde=True,
                 hist_kws={'histtype':'step','linewidth':1,'alpha':0.85}, kde_kws={'linewidth':0}, ax=axParticle36, label=file)
    if file != 'non07-11':
        sns.distplot(dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce']/1e6, bins=int(dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce'].max()/bw), hist=True, kde=True,
                     hist_kws={'histtype':'step','linewidth':1,'alpha':0.85}, kde_kws={'linewidth':0}, ax=axPins36, label=file)
for file in ['squ64_07-12','tri64_07-12','non07-11']:
    if file != 'non07-11':
        sns.distplot(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce']/1e6, bins=int(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce'].max()/bw), hist=True, kde=True,
                     hist_kws={'histtype':'step','linewidth':1,'alpha':0.85}, kde_kws={'linewidth':0}, ax=axParticle64, label=file)
        sns.distplot(dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce']/1e6, bins=int(dataDict[file].loc[dataDict[file].bondType>=3,'normalizedForce'].max()/bw), hist=True, kde=True,
                     hist_kws={'histtype':'step','linewidth':1,'alpha':0.85}, kde_kws={'linewidth':0}, ax=axPins64, label=file)
    else:
        sns.distplot(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce']/1e6, bins=int(dataDict[file].loc[dataDict[file].bondType<=2,'normalizedForce'].max()/bw), color='C3', hist=True, kde=True,
                     hist_kws={'histtype':'step','linewidth':1,'alpha':0.85}, kde_kws={'linewidth':0}, ax=axParticle64, label=file)
axParticle36.legend()
axParticle36.set_xlim(-0.2,18)
axParticle36.set_ylim(3e-5,1)
axParticle36.set_yscale('log')

axPins36.legend()
axPins36.set_xlim(-0.2,18)
axPins36.set_ylim(3e-5,1)
axPins36.set_yscale('log')

axParticle64.legend()
axParticle64.set_xlim(-0.2,18)
axParticle64.set_ylim(3e-5,1)
axParticle64.set_yscale('log')

axPins64.legend()
axPins64.set_xlim(-0.2,18)
axPins64.set_ylim(3e-5,1)
axPins64.set_yscale('log')

axParticle36.set_title('particle-particle contacts')
axPins36.set_title('particle-pin contacts')
axParticle36.set_xlabel('')
axParticle64.set_xlabel('')
axPins36.set_xlabel('')
axPins64.set_xlabel('')

fig.tight_layout()
plt.subplots_adjust(left=0.07,bottom=0.05,top=0.94, right=0.978)
fig.suptitle('particle-particle and particle-pin force distributions, July 2020 data', fontsize=14)
fig.text(0.5, 0.01, 'normalized force', ha='center')
fig.text(0.02, 0.5, 'probability density', va='center', rotation='vertical')
fig.text(0.98, 0.73, '36 pins', va='center', rotation=-90, fontsize=12)
fig.text(0.98, 0.27, ' 64 pins', va='center', rotation=-90, fontsize=12)
#plt.savefig('plots/force_distribution_bondType_numPins-comparison-07_2020.png', dpi=300)
plt.show()

## Analysis of Weak Particle-Pin Contacts

In [ ]:
weakPins=dataDict['squ100_07-19'][(dataDict['squ100_07-19']['normalizedForce']<5000)&(dataDict['squ100_07-19']['bondType']>=3)].copy()
weakPins['contactsParticle']=weakPins.apply(lambda x: contactsDict['squ100_07-19'].loc[x.name[0],x['x2'],x['y2']]['contacts'], axis=1)
weakPins['contactsPin']=weakPins.apply(lambda x: contactsDict['squ100_07-19'].loc[x.name[0],x['x1'],x['y1']]['contacts'], axis=1)
weakPins[weakPins.contactsParticle==7].sort_values('normalizedForce').head(30)

In [ ]:
plot_topography('squ36_07-11', 100, savefig='plots/presentation/squ36_demo')

In [ ]:
plt.figure(figsize=(8,5))
for file in files:
    plt.hist(dataDict[file][dataDict[file].bondType>=3].groupby(level=0)['normalizedForce'].nsmallest(2)*1e-6, bins=10**np.linspace(-6,0,13), histtype='step', label=file)
plt.axvline(5e-3,alpha=0.5)
plt.title('normalized force of two weakest particle-pin bonds per seed')
plt.xlabel('normalized force')
plt.ylabel('counts')
plt.xscale('log')
plt.legend(loc='upper left')
#plt.savefig('plots/two_smallest_pf_normalized_july-comparison.png', dpi=300)

In [ ]:
contactsDict={file:sort_particles(dataDict[file]).groupby(['x','y','trialName']).size().reset_index(name='contacts') for file in (files)} # dataframe of particle coordinates, with contact number and trial labeled
for file in tqdm_notebook(files):
    contactsDict[file].set_index(['trialName','x','y'], inplace=True)
    weakPins=dataDict[file][(dataDict[file].bondType>=3)&(dataDict[file].normalizedForce<=5e3)][['x1','y1','x2','y2','normalizedForce']].copy()
    weakPins['contactsParticle']=weakPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x2'],x['y2']]['contacts'], axis=1)
    weakPins['contactsPin']=weakPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x1'],x['y1']]['contacts'], axis=1)
    allPins=dataDict[file][(dataDict[file].bondType>=3)][['x1','y1','x2','y2','normalizedForce']].copy()
    allPins['contactsParticle']=allPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x2'],x['y2']]['contacts'], axis=1)
    allPins['contactsPin']=allPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x1'],x['y1']]['contacts'], axis=1)
    fig, (axAll, axWeak) = plt.subplots(1,2, figsize=(12,4))
    for ax, data in [(axAll, allPins), (axWeak, weakPins)]:
        ax.hist(data['contactsParticle'],bins=np.linspace(2.5,7.5,6), density=True, label='particle contacts')
        ax.hist(data['contactsPin'],bins=np.linspace(0.5,2.5,3), density=True, label = 'pin contacts')
        ax.set_ylim(0,0.9)
        ax.set_axisbelow(True)
        ax.grid(linestyle='dashed')
        ax.set_xlabel('number of contacts')
    axWeak.set_title('weak contacts, normalized force < 0.005')
    axAll.set_title('all contacts')
    axWeak.legend()
    fig.subplots_adjust(top=0.85)
    plt.suptitle('normalized distribution of particle/pin contacts, {}'.format(file), fontsize=14, x=0.52)
    #plt.savefig('plots/particlePinContactNumbers/particle_pin_contact_distribution-{}.png'.format(file),dpi=300)
    plt.show()

In [ ]:
allDF = pd.DataFrame()
weakDF = pd.DataFrame()
for file in tqdm_notebook(files):
    contactsDict={file:sort_particles(dataDict[file]).groupby(['x','y','trialName']).size().reset_index(name='contacts') for file in (files)} # dataframe of particle coordinates, with contact number and trial labeled
    contactsDict[file].set_index(['trialName','x','y'], inplace=True)
    weakPins=dataDict[file][(dataDict[file].bondType>=3)&(dataDict[file].normalizedForce<=5e-3)][['x1','y1','x2','y2','normalizedForce']].copy()
    weakPins['contactsParticle']=weakPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x2'],x['y2']]['contacts'], axis=1)
    weakPins['contactsPin']=weakPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x1'],x['y1']]['contacts'], axis=1)
    allPins=dataDict[file][(dataDict[file].bondType>=3)][['x1','y1','x2','y2','normalizedForce']].copy()
    allPins['contactsParticle']=allPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x2'],x['y2']]['contacts'], axis=1)
    allPins['contactsPin']=allPins.apply(lambda x: contactsDict[file].loc[x.name[0],x['x1'],x['y1']]['contacts'], axis=1)
    allDF = pd.concat([allDF,allPins[['contactsParticle', 'contactsPin']]])
    weakDF = pd.concat([weakDF,weakPins[['contactsParticle','contactsPin']]])

In [ ]:
fig, (axAll, axWeak) = plt.subplots(1,2, figsize=(12,4))
for ax, data in [(axAll, allDF), (axWeak, weakDF)]:
    ax.hist(data['contactsParticle'],bins=np.linspace(2.5,7.5,6), density=True, label='particle contacts')
    ax.hist(data['contactsPin'],bins=np.linspace(0.5,2.5,3), density=True, label = 'pin contacts')
    ax.set_ylim(0,0.9)
    ax.set_axisbelow(True)
    ax.grid(linestyle='dashed')
    ax.set_xlabel('number of contacts')
axWeak.set_title('weak contacts, normalized force < 0.005')
axAll.set_title('all contacts')
axWeak.legend()
fig.subplots_adjust(top=0.85)
plt.suptitle('normalized distribution of particle/pin contacts, aggregated', fontsize=14, x=0.52)
#plt.savefig('plots/particlePinContactNumbers/particle_pin_contact_distribution-aggregate.png',dpi=300)
#plt.close()

# 2. Analysis on Single Trial

## 2.1. Topology Plotting Function
The plot_topography function takes a DataFrame and name of a trial and plots it. This primarily consists of preparing BondInfo data for plotting, which entails sorting particles by type, accounting for bonds which wrap at the boundary, and if the data is of a triangular lattice, applying a transformation before and after the other steps to make it compatible with our square lattice process.

This is necessary as our triangle lattice data is provided with rhombus rather than square boundaries, so we need to pivot the data $30^\circ$ to the left with respect to the x-axis to transform it into a square, account for wrapping, then pivot it back.

Note that pins are drawn disproportionately large for reference, and since we're only given the small particle radius, the large radius is found by scaling the small radius up by a factor of 1.4. If this relation varies, we will need more information.

In [ ]:
plot_topography('non0_08-02',80, show=True, particles=True, pins=False, savefig='plots/presentation/jammed')

In [ ]:
from matplotlib import collections
from matplotlib.lines import Line2D

# What's really important: picking a fun color scheme
#colormap = matplotlib.cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 'r', '#FFFFFF', '#FFFFFF']) # classic
colormap = matplotlib.cycler('color',['#FAD55C','#68C182','#267DB3','#ED6647','#8561C8','#000000', '#fef9e6', '#c0dff2']) # blue-yellow
plt.rc('axes',prop_cycle=colormap)

def plot_topography(file, trialName, savefig=False, title='Default', particles=True, pins=True, bonds=True, show=True, cutoff=None, sheared=True):
    # PARAMETERS:
    # file (str) - abbreviated file name, as specified list of files in first cell
    # trialName (str or int) - full trial name, e.g. 'ran-90440-266-36-0.02781-.16667', or integer index of a certain trial
    # savefig (bool/str) - If True, saves to plots folder. If string, saves to specified path.
    # title (str) - name for figure, if not default.
    # particles (bool) - whether or not to show particles.
    # pins (str) - if "all", attempts to extrapolate the entire pin lattice from existing pins. If False, pins are not shown. Otherwise, plots only contacted pins.
    # sheared (bool) - only significant for triangular lattices: if False, plots the data as a square instead ofa rhombus.
    # show (bool) - whether or not display the figure in output.
    # cutoff (float) - value at which to filter out forces. If positive, only show forces greater than the magnitude, and if negative, only show forces less than the magnitude.

    if isinstance(trialName,int):
        trialName=dataDict[file].index.get_level_values('trialName').unique()[trialName]

    bondInfo=dataDict[file].loc[trialName].copy() # get the data
    #find the mean force before filtering out bonds weaker than the cutoff
    meanForce=bondInfo.force.abs().mean()
    if cutoff:
        if cutoff>0:
            bondInfo=bondInfo[bondInfo.normalizedForce>=np.abs(cutoff)]
        elif cutoff<0:
            bondInfo=bondInfo[bondInfo.normalizedForce<=np.abs(cutoff)]

    bondInfo['bondColor']=bondInfo['bondType'].transform(lambda x: 'C'+str(x)) # takes bondtypes and transforms them into color codes

    headerNames=['geometry','seed','num','pins','small_radius','pin_separation']
    trialInfo={}
    for key, label in zip(headerNames,trialName.split('-')):
        trialInfo[key]=label

    # MAKING COORDINATE DATA USEFUL
    all_particles = sort_particles(bondInfo)[['x','y','particleType']].drop_duplicates() # get dataframe of particle coordinates sorted by bond type
    all_particles[['x','y']]=all_particles[['x','y']]*1e-6
    bondInfo[['x1','y1','x2','y2','normalizedForce']]=bondInfo[['x1','y1','x2','y2','normalizedForce']]*1e-6
    if trialInfo['geometry']=='tri': # extra steps for triangular lattice of pins
        # Un-shear the data, so that we can apply the same wrapping algorithm as in the square
        # unshearing matrix: [[1, -tan(30)],[0,1+tan(30)*tan(15)]] (in degrees)
        for x,y in [('x1','y1'),('x2','y2')]: # apply transformation to both sets of coordinate columns
            bondInfo[x]=bondInfo[x]-bondInfo[y]*np.tan(np.pi/6)
            bondInfo[y]=bondInfo[y]*(1+np.tan(np.pi/6)*np.tan(np.pi/12))

        if sheared==False: # normally we don't transform the particles at all, but if we want an unsheared plot we need to unshear the particles as well
            all_particles['x']-=all_particles['y']*np.tan(np.pi/6)
            all_particles['y']+=all_particles['y']*np.tan(np.pi/6)*np.tan(np.pi/12)

    # Find wrapping contacts - bonds that are too long not to be wrapping
    # (this is not memory efficient, but individual trials are small enough that we don't care)
    wrapsX=bondInfo[np.abs(bondInfo.x1-bondInfo.x2)>=0.5].copy() # 0.5 is kind of arbitrary here, all that matters is distance > (R_A+R_B)
    wrapsY=bondInfo[np.abs(bondInfo.y1-bondInfo.y2)>=0.5].copy()
    internalBonds=bondInfo.drop(pd.concat([wrapsX,wrapsY]).index.values) # dataframe of non-wrapped contacts

    # Wrapping algorithm: depending on which axis we're wrapping around, signs will flip - this value gives 1 or -1 for each wrapped contact according to sign convention
    wrapsY['yAdjust']=round(wrapsY['y1']-wrapsY['y2'])
    wrapsX['xAdjust']=round(wrapsX['x1']-wrapsX['x2'])

    # Additional case for when bonds wrap through three regions via corner
    wrapsCorner=pd.merge(wrapsX,wrapsY) # dataframe of contacts which wrap on X and Y, with both xAdjust and yAdjust included (corner cases have two sign conventions)
    cornerIndex=wrapsX.index.intersection(wrapsY.index) # TODO: probably a more optimal way to do this, since we're effectively checking for duplicates twice on this line and the previous.

    # Separate corner cases from edge cases (they'll plot differently)
    wrapsX=wrapsX.drop(cornerIndex)
    wrapsY=wrapsY.drop(cornerIndex)

    # Reformat data for LineCollection plotting - TODO: optimize
    # Internal lines
    sourcePoints=np.array([internalBonds['x1'],internalBonds['y1']]).T.reshape(-1,1,2)
    targetPoints=np.array([internalBonds['x2'],internalBonds['y2']]).T.reshape(-1,1,2)
    internalsegments=np.concatenate([sourcePoints,targetPoints],axis=1)

    # X-wrapped lines
    # don't love the repetitive .values calls here, but in 1- or 0- length cases this preserves the shape of the array
    sourcePoints=np.array([wrapsX['x1'].values-wrapsX['xAdjust'].values,wrapsX['y1'].values])
    targetPoints=np.array([wrapsX['x2'].values,wrapsX['y2'].values])
    sourcePoints=np.concatenate([sourcePoints,np.array([wrapsX['x1'].values,wrapsX['y1'].values])]).T.reshape(-1,1,2)
    targetPoints=np.concatenate([targetPoints,np.array([wrapsX['x2'].values+wrapsX['xAdjust'].values,wrapsX['y2'].values])]).T.reshape(-1,1,2)
    xsegments=np.concatenate([sourcePoints,targetPoints],axis=1)

    # Y-wrapped lines
    sourcePoints=np.array([wrapsY['x1'].values,wrapsY['y1'].values-wrapsY['yAdjust'].values])
    targetPoints=np.array([wrapsY['x2'].values,wrapsY['y2'].values])
    sourcePoints=np.concatenate([sourcePoints,np.array([wrapsY['x1'].values,wrapsY['y1'].values])]).T.reshape(-1,1,2)
    targetPoints=np.concatenate([targetPoints,np.array([wrapsY['x2'].values,wrapsY['y2'].values+wrapsY['yAdjust'].values])]).T.reshape(-1,1,2)
    ysegments=np.concatenate([sourcePoints,targetPoints],axis=1)

    # Corner-wrapped lines
    sourcePoints=np.array([]).reshape(-1,1,2)
    targetPoints=np.array([]).reshape(-1,1,2)
    for i in [0,1]: # 2x2 loop (one run for each corner)
        for j in [0,1]:
            sourcePoints=np.concatenate([sourcePoints,np.array([wrapsCorner['x1'].values+wrapsCorner['xAdjust'].values*(i-1),
                                                                wrapsCorner['y1'].values+wrapsCorner['yAdjust'].values*(j-1)]).T.reshape(-1,1,2)])
            targetPoints=np.concatenate([targetPoints,np.array([wrapsCorner['x2'].values+wrapsCorner['xAdjust'].values*(i),
                                                                wrapsCorner['y2'].values+wrapsCorner['yAdjust'].values*(j)]).T.reshape(-1,1,2)])
    cornersegments=np.concatenate([sourcePoints,targetPoints],axis=1)

    if trialInfo['geometry']=='tri' and sheared==True:
        # re-shear the data
        # shearing matrix is [[1,sin(30)],[0,1-sin(30)*tan(15)]] (in degrees)
        for segmentgroup in [internalsegments,xsegments,ysegments,cornersegments]: # bad form to iterate like this, but like above the arrays are small enough to get away with it
            for segment in segmentgroup:
                for point in segment:
                    point[0]+=point[1]*np.sin(np.pi/6)
                    point[1]-=point[1]*np.sin(np.pi/6)*np.tan(np.pi/12)

    # Initialize patch collections
    # lines are plotted with thickness normalizedForce+1 to scale with bond strength while all being visible
    internalLines=collections.LineCollection(internalsegments,linewidths=internalBonds['normalizedForce']+1,color=internalBonds['bondColor'])
    xLines=collections.LineCollection(xsegments,linewidths=wrapsX['normalizedForce'].repeat(2)+1,color=wrapsX['bondColor'].repeat(2))
    yLines=collections.LineCollection(ysegments,linewidths=wrapsY['normalizedForce'].repeat(2),color=wrapsY['bondColor'].repeat(2))
    cornerLines=collections.LineCollection(cornersegments,linewidths=wrapsCorner['normalizedForce'].repeat(4),color=wrapsCorner['bondColor'].repeat(4))

    particleList=[]
    if particles:
        particleList+=[plt.Circle((x,y), radius=float(trialInfo['small_radius'])*1.4,ec='C5',fc='C7') for x,y in all_particles.loc[all_particles.particleType==2,['x','y']].values]
        particleList+=[plt.Circle((x,y), radius=float(trialInfo['small_radius']),ec='C5',fc='C6') for x,y in all_particles.loc[all_particles.particleType==1,['x','y']].values]
    if pins=='all' and trialInfo['geometry']!='ran':
        allPinsInFile=sort_particles(dataDict[file],types=[0])
        pinMin=allPinsInFile.min()*1e-6
        pinMax=allPinsInFile.max()*1e-6
        if trialInfo['geometry']=='tri':
            pinMin['x']-=pinMin['y']*np.tan(np.pi/6)
            pinMax['x']-=pinMax['y']*np.tan(np.pi/6)
        pinGrid=np.meshgrid(np.linspace(pinMin['x'],pinMax['x'],int(np.sqrt(int(trialInfo['pins'])))),np.linspace(pinMin['x'],pinMax['x'],int(np.sqrt(int(trialInfo['pins'])))))
        if trialInfo['geometry']=='tri' and sheared==True:
            pinGrid[0]+=pinGrid[1]*np.sin(np.pi/6)
            pinGrid[1]-=pinGrid[1]*np.sin(np.pi/6)*np.tan(np.pi/12)
        particleList+=[plt.Circle((x,y), radius=0.005, color='k',fill=True) for x,y in np.vstack([pinGrid[0].flatten(),pinGrid[1].flatten()]).T]
    elif pins==True:
        particleList+=[plt.Circle((x,y), radius=0.005, color='k',fill=True) for x,y in all_particles.loc[all_particles.particleType==0,['x','y']].values]

    ParticleCollection=collections.PatchCollection(particleList, match_original=True)

    plt.figure(figsize=(8,8)) # large, square plot
    a = plt.subplot() # initialize axes

    # Plot lines
    a.set(aspect='equal') # plot axes to scale
    if bonds:
        a.add_collection(internalLines)
        a.add_collection(xLines)
        a.add_collection(yLines)
        a.add_collection(cornerLines)

    # Plot particles
    a.add_collection(ParticleCollection)

    # Cosmetics
    if trialInfo['geometry']=='tri' and sheared==True: # for the rhombus, we need different axis limits
        a.set_xlim(0,1.5)
        a.set_ylim(-0.1,np.sqrt(3)/2+0.1)
        legendLocation='lower right'
    else:
        a.set_xlim(0,1)
        a.set_ylim(0,1)
        legendLocation='upper right'
    a.set_xlabel('x')
    a.set_ylabel('y')
    if title=='Default':
        a.set_title('Topography of {} data, {} total, seed {}, mean force={mf:.4f}'.format(file, *[trialInfo[x] for x in ['num','seed']], mf=meanForce))
    else:
        a.set_title(title)

    # Particle legend
    # can't directly label collections, so we designate shapes just for the legends
    a.scatter([],[],facecolors='C7',edgecolors='C5', s=150, label='large grains')
    a.scatter([],[],facecolors='C6',edgecolors='C5', s=100, label = 'small grains')
    a.scatter([],[],c='k', s=30,label='pins')

    particleLegend1 = a.legend(loc=legendLocation,title='Particle Types',framealpha=0.95)
    a.add_artist(particleLegend1) # matplotlib also overwrites with the latest legend call by default, so we specify we want this one as-is

    # Line legend
    custom_lines1 = [Line2D([0], [0], color='C0', lw=2),
                    Line2D([0], [0], color='C1', lw=2),
                    Line2D([0], [0], color='C2', lw=2),
                    Line2D([0], [0], color='C3', lw=2),
                    Line2D([0], [0], color='C4', lw=2)]
    lineLegend = a.legend(custom_lines1,['small-small','small-large','large-large','pin-small','pin-large'],loc='upper left',title='Contact Types',framealpha=0.95)

    if savefig:
        if isinstance(savefig,str):
            if not os.path.exists('/'.join(savefig.split('/')[:-1])):
                os.makedirs('/'.join(savefig.split('/')[:-1]))
            if cutoff:
                saveName = '{}-cutoff_{}.png'.format(savefig, np.abs(cutoff))
            else:
                saveName = '{}.png'.format(savefig)
        else:
            saveName = 'plots/{}.png'.format(trialName)
            print('Directory not specified, saving to plots folder')
        plt.savefig(saveName, dpi=300)
    if show:
        plt.show()
    plt.close()
    try:
        return saveName
    except:
        return trialName

In [ ]:
for file in files:
    plot_topography(file, 0, pins='all', particles=False, sheared=False, savefig='plots/forAmy_allPins', show=True)

## 2.3. Animating Increasing Bond Strength Cutoff for a Trial/Identifying and Plotting Exceptionally Large Forces

In [ ]:
import imageio
import os

for file in tqdm_notebook(['squ100_07-19']):
    print('FILE:', filenames[file])
    # Plotting exceptions
    for trialName in tqdm_notebook(dataDict[file]['normalizedForce'].nlargest(10).index.get_level_values(0).unique()):
        plot_topography(file, trialName, savefig='plots/exceptions', title='{} exception case, trial name {}'.format(file, trialName))

    # Creating Animations
    #for trialName in tqdm_notebook(dataDict[file]['normalizedForce'].nlargest(10).index.get_level_values(0).unique()):
    #    image_list=[]
    #    if not os.path.exists('Bond_Animations/'+trialName):
    #        os.makedirs('Bond_Animations/'+trialName)
    #    print('TRIAL:', trialName)
    #    for cutoffForce in tqdm_notebook([-100.0,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1]):
    #        saveName=plot_topography(file, trialName, savefig='Bond_Animations/'+trialName, cutoff=cutoffForce, title='{} trial name {}, force cutoff {}'.format(file, trialName, np.abs(cutoffForce)))
    #        image_list.append(imageio.imread(saveName))
    #    imageio.mimwrite('Bond_Animations/{}/animated.gif'.format(trialName), image_list, duration = 2)

In [ ]:
dataDict['tri100_07-19']['normalizedForce'].nlargest(40).index.get_level_values(0).unique()

## 2.4. (Deprecated). Plotting lines directly in MatPlotLib
This code is fast, but plt.plot() can't account for varying line width; we end up using a LineCollection to do this in the next cell. Leaving this here for posterity.

## 2.5. (Deprecated). Plot with particleinfo.txt
A sample plotting of data from the last cell, with particles extrapolated from simulation_data/june_18/squ_BondInfo.txt, compared against a plot with particle types and locations explicitly given by a particleinfo.txt file. Note that with an additional file, we can pinpoint the location of rattlers and untouched pins.

## 2.6 (Deprecated). Plot contact force distribution as histogram for a single trial
Plots normalized contact forces $\frac{f}{\langle f\rangle}$ using seaborn's distplot() function. At this point, the y axis is not normalized; the kde parameter allows for normalization for a single data set, but to normalize the second plot collectively appears challenging.